# 使用领域（私有）数据微调 ChatGLM3

生成带有 epoch 和 timestamp 的模型文件

In [1]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [2]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
train_data_path = 'data/zhouyi_dataset_20240118_163659.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

## 数据处理

In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,坤卦和启蒙教育之间有何联系？,坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。
1,周易的坤卦讲述了什么？,坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。
2,比卦是怎样的一个卦象？,在周易中，比卦代表着相亲相依的意涵，它是由上卦坎（水）和下卦坤（地）相叠而成。这一卦象预示着长期的吉利和无咎，同时也暗示着不愿臣服的邦国迟迟不来朝会有难。\n\n比卦的核心哲学是：水附大地，地纳河海，象征相亲相依，亲密无间，展示出宽宏无私，精诚团结的道理。因此，在这一卦象中，人们可以得到贵人的提拔，事业可望成功，但需诚实、信任地做事，待人宽厚、正直，主动热情。在经商中，也需真诚交往，遵守商业道德，不可贪心不足或自以为是。\n\n比卦的运势平顺，事业顺利可望成功，可得贵人提拔。经商方面，愿望能够实现且有利润，但需与他人密切合作，讲究商业道德。在婚恋方面，象征着美好姻缘和相亲相爱。在决策中，建议心地善良，待人忠诚、厚道，工作勤恳并善于选择朋友。
3,比卦和启蒙教育之间有何联系？,在周易中，比卦代表着相亲相依的意涵，它是由上卦坎（水）和下卦坤（地）相叠而成。这一卦象预示着长期的吉利和无咎，同时也暗示着不愿臣服的邦国迟迟不来朝会有难。\n\n比卦的核心哲学是：水附大地，地纳河海，象征相亲相依，亲密无间，展示出宽宏无私，精诚团结的道理。因此，在这一卦象中，人们可以得到贵人的提拔，事业可望成功，但需诚实、信任地做事，待人宽厚、正直，主动热情。在经商中，也需真诚交往，遵守商业道德，不可贪心不足或自以为是。\n\n比卦的运势平顺，事业顺利可望成功，可得贵人提拔。经商方面，愿望能够实现且有利润，但需与他人密切合作，讲究商业道德。在婚恋方面，象征着美好姻缘和相亲相爱。在决策中，建议心地善良，待人忠诚、厚道，工作勤恳并善于选择朋友。
4,蒙卦在周易哲学中扮演什么角色？,蒙卦是由艮卦（山）下，坎卦（水）上组成的异卦相叠。它代表着通泰，启蒙的意义。在这里，卜者并非是在向幼稚愚昧的人取求，而是幼稚愚昧的人在向卜者求教。第一次卜筮就得到了神灵的指示。然而，如果轻慢不敬地再三卜筮的话，神灵便不会再示警。总的来说，这是一个吉利的卜问。\n\n蒙卦的核心在于山下有泉的形象，寓意着启蒙。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德，像山泉一样果断行动。然而，此卦乃是离宫四世卦，它代表着回还往复、疑惑不前、多忧愁过失，因而属于凶卦。\n\n蒙卦在个人发展、事业经商、求名婚恋等方面的解释不一。在事业方面，表示事业初建，具有启蒙和通达之象，需要勇敢坚毅的行动；而在经商方面，需要务必小心谨慎，树立高尚的商业道德，不可急功近利；求名方面，需要接受良好的基础教育，陶冶情操。整体而言，此卦提示须忍耐待机而动，听取别人意见，方能通达运势。


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

In [7]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [8]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

In [9]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

In [10]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [11]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

## 加载模型

In [29]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- modeling_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ValueError: The model class you are passing has a `config_class` attribute that is not consistent with the config class you passed (model has <class 'transformers_modules.THUDM.chatglm3-6b.f30825950ce00cb0577bf6a15e0d95de58e328dc.configuration_chatglm.ChatGLMConfig'> and you passed <class 'transformers_modules.THUDM.chatglm3-6b.37f2196f481f8989ea443be625d05f97043652ea.configuration_chatglm.ChatGLMConfig'>. Fix one of those so they match!

In [13]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [14]:
target_modules

['query_key_value']

In [15]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [16]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


### QLoRA 微调模型

In [17]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 3
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=8,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [19]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
trainer.train()

/root/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.672500
2,4.701000
3,4.274900
4,4.403600
5,3.959300
6,3.692300
7,3.526800
8,3.122000
9,3.005800
10,2.740100


/root/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.p

TrainOutput(global_step=60, training_loss=0.9092419001429032, metrics={'train_runtime': 393.9197, 'train_samples_per_second': 1.219, 'train_steps_per_second': 0.152, 'total_flos': 4623664398827520.0, 'train_loss': 0.9092419001429032, 'epoch': 3.0})

In [21]:
trainer.model.save_pretrained(output_dir)

# homework:基于 data 目录下的数据训练 ChatGLM3 模型，使用 inference Notebook 对比微调前后的效果。

### 配置环境

In [1]:
#安装必要依赖环境，只需要运行一次
!pip install -r ../requirements.txt

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.9 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 13.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 14.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 13.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
pip install protobuf transformers==4.30.2 cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate

Note: you may need to restart the kernel to use updated packages.


In [1]:
#配置缓存及网络环境，只需要开始时运行一次
import os
#在transformers自定义模型下载的路径方法
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["HF_DATASETS_CACHE"] = "../../autodl-tmp/datasets_cache/"
os.environ["HF_HOME"] = "../../autodl-tmp/cache/"
os.environ["HUGGINGFACE_HUB_CACHE"] = "../../autodl-tmp/hub_cache/"

#配置网络环境，只需要开始时运行一次

import subprocess
import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True, encoding='utf-8')

output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# 验证环境配置是否修改成功
print("http_proxy",os.environ.get("http_proxy"))
print("https_proxy",os.environ.get("https_proxy"))
print("HF_HOME",os.environ.get("HF_HOME"))
print("HF_DATASETS_CACHE",os.environ.get("HF_DATASETS_CACHE"))
print("HUGGINGFACE_HUB_CACHE",os.environ.get("HUGGINGFACE_HUB_CACHE"))

http_proxy http://172.20.0.113:12798
https_proxy http://172.20.0.113:12798
HF_HOME ../../autodl-tmp/cache/
HF_DATASETS_CACHE ../../autodl-tmp/datasets_cache/
HUGGINGFACE_HUB_CACHE ../../autodl-tmp/hub_cache/


In [2]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

### 全局变量

In [3]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
train_data_path = 'data/zhouyi_dataset_20240118_163659.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

### 数据处理

In [4]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


In [5]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,周易比卦的核心思想是什么？,在周易中，比卦代表着相亲相依的意涵，它是由上卦坎（水）和下卦坤（地）相叠而成。这一卦象预示着长期的吉利和无咎，同时也暗示着不愿臣服的邦国迟迟不来朝会有难。\n\n比卦的核心哲学是：水附大地，地纳河海，象征相亲相依，亲密无间，展示出宽宏无私，精诚团结的道理。因此，在这一卦象中，人们可以得到贵人的提拔，事业可望成功，但需诚实、信任地做事，待人宽厚、正直，主动热情。在经商中，也需真诚交往，遵守商业道德，不可贪心不足或自以为是。\n\n比卦的运势平顺，事业顺利可望成功，可得贵人提拔。经商方面，愿望能够实现且有利润，但需与他人密切合作，讲究商业道德。在婚恋方面，象征着美好姻缘和相亲相爱。在决策中，建议心地善良，待人忠诚、厚道，工作勤恳并善于选择朋友。
1,蒙卦在周易中代表了哪些方面？,蒙卦是由艮卦（山）下，坎卦（水）上组成的异卦相叠。它代表着通泰，启蒙的意义。在这里，卜者并非是在向幼稚愚昧的人取求，而是幼稚愚昧的人在向卜者求教。第一次卜筮就得到了神灵的指示。然而，如果轻慢不敬地再三卜筮的话，神灵便不会再示警。总的来说，这是一个吉利的卜问。\n\n蒙卦的核心在于山下有泉的形象，寓意着启蒙。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德，像山泉一样果断行动。然而，此卦乃是离宫四世卦，它代表着回还往复、疑惑不前、多忧愁过失，因而属于凶卦。\n\n蒙卦在个人发展、事业经商、求名婚恋等方面的解释不一。在事业方面，表示事业初建，具有启蒙和通达之象，需要勇敢坚毅的行动；而在经商方面，需要务必小心谨慎，树立高尚的商业道德，不可急功近利；求名方面，需要接受良好的基础教育，陶冶情操。整体而言，此卦提示须忍耐待机而动，听取别人意见，方能通达运势。
2,坤卦是怎样的一个卦象？,坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。
3,周易乾卦的核心思想是什么？,在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。\n\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。
4,蒙卦和教育启蒙有什么联系？,蒙卦是由艮卦（山）下，坎卦（水）上组成的异卦相叠。它代表着通泰，启蒙的意义。在这里，卜者并非是在向幼稚愚昧的人取求，而是幼稚愚昧的人在向卜者求教。第一次卜筮就得到了神灵的指示。然而，如果轻慢不敬地再三卜筮的话，神灵便不会再示警。总的来说，这是一个吉利的卜问。\n\n蒙卦的核心在于山下有泉的形象，寓意着启蒙。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德，像山泉一样果断行动。然而，此卦乃是离宫四世卦，它代表着回还往复、疑惑不前、多忧愁过失，因而属于凶卦。\n\n蒙卦在个人发展、事业经商、求名婚恋等方面的解释不一。在事业方面，表示事业初建，具有启蒙和通达之象，需要勇敢坚毅的行动；而在经商方面，需要务必小心谨慎，树立高尚的商业道德，不可急功近利；求名方面，需要接受良好的基础教育，陶冶情操。整体而言，此卦提示须忍耐待机而动，听取别人意见，方能通达运势。


In [7]:
# 我的软件环境：
# Python=3.11
# torch==2.1.2
# transformers==4.36.1

# 今天重跑“qlora_chatglm3_timestamp.ipynb”文件时，发现爆显存了，同样的运行环境，10几天前运行时是没有问题的。感觉不是环境问题，也不是老师代码的问题。查看运行日志，发现huggingface拉取了最新版本的chatglm3-6b（revision='f308259'），然后查看新版本chatglm3-6b的commit代码改动，发现torch.utils.checkpoint.checkpoint()方法的参数有改动，添加了“use_reentrant=False”，而在PyTorch=2.1.2版本中，checkpoint()方法的“use_reentrant”参数值默认为True。

# 新版本chatglm3-6b的commit代码改动：
# https://huggingface.co/THUDM/chatglm3-6b/commit/37fe0008ee928af7f4e5e57693e8c7787d049af8

# 查看PyTorch文档发现：
# use_reentrant=True 时，在前向传播过程中不会保存“autograd graph”。
# use_reentrant=False 时，则会保存“autograd graph”，会占用更多的显存。
# 未来的PyTorch版本会把“use_reentrant”参数默认值设置为False，目前版本（2.1.2）的默认值为True。

# 使用最新版本的chatglm3-6b做微调时爆内存，估计就是因为“use_reentrant”参数改为False而引起的。微调加载模型时，指定一个旧版本号就可以了。代码例子如下：
# tokenizer = AutoTokenizer.from_pretrained(......, revision='b098244')
# model = AutoModel.from_pretrained(......, revision='b098244')

# 指定旧版本号后，重跑老师的代码，没再发现爆显存问题了。如果大家有更好的方法，欢迎互相交流。

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True,revision='b098244', use_reentrant=True)

tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

In [8]:
# tokenize_func 函数

# 当微调模型进行问答任务时，我们的目标是让模型能够正确地根据给定的问题来预测答案。
# 在实际应用中，用户提出问题时并不会提供答案，而是期望模型能够根据问题和其理解的语境来生成合理的答案。
# 因此，在微调过程中，我们希望模型能够学习到问题和答案之间的语义关系和逻辑关联，以便在实际应用中更准确地进行答案预测。

# 如果我们将答案作为标签，而不将问题和答案一起作为输入，那么在微调过程中，模型将只能通过问题来学习到答案的内容，而无法学习到问题和答案之间的关系。
# 这可能会导致模型在预测答案时过于依赖问题本身，而忽略了问题与答案之间的语义和逻辑联系。
# 因此，模型可能会产生不准确的答案，因为它没有充分考虑到问题的信息。

# 通过将问题和答案一起作为输入，并将答案作为标签，我们可以确保模型在微调过程中能够充分地考虑到问题的信息，并学习到问题和答案之间的关联。
# 这样，模型就能够更好地理解问题的语义，并根据问题生成合理的答案。因此，这种设计可以提高模型在实际问答任务中的性能和准确度。


def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    
    # 如果样本 example 中存在 'input' 键并且对应的值不是空白字符串，则执行下面的代码块。
    # 在执行代码块时，会将 'input' 对应的值添加到问题文本 question 的末尾，并在其前面添加一个换行符，以便将问题文本和输入文本分隔开来。
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    # 在使用 Transformers 库时，一般会在输入序列的开头和结尾添加特殊标记，以便模型能够正确地理解输入的语境。
    # 这些特殊标记通常包括起始标记（如 [CLS]）、分隔标记（如 [SEP]）等。
    # build_inputs_with_special_tokens 函数会将问题序列和答案序列合并，并在适当的位置添加特殊标记，以构建出模型可以接受的输入序列。
    # 在这段代码中，q_ids 是问题序列的 token IDs，a_ids 是答案序列的 token IDs。
    # 通过调用 tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)，会将问题和答案序列合并，并添加特殊标记，最终得到模型可接受的输入序列。
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    # 在模型训练过程中，通常会采用类似于序列到序列（Sequence-to-Sequence，Seq2Seq）的方式。
    # 在这种情况下，需要为模型提供输入序列和相应的标签序列，以便模型学习将输入序列转换为标签序列。
    # 在这段代码中，首先使用 ignore_label_id 填充了问题部分的标签序列，填充的长度为问题部分的长度 question_length。
    # 然后将输入序列 input_ids 的剩余部分（即答案部分）加入到标签序列中，这样就得到了模型的标签序列。
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [9]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [10]:
# **shuffle(seed=seed)：**这个操作是对数据集进行随机重排，即打乱数据集中样本的顺序。
# 参数seed是随机种子，用于控制随机化过程的可重复性，保证在相同种子下每次运行得到的结果是一样的。
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)

# **flatten_indices()：**这个操作是将数据集的索引重新排序，确保后续的批处理过程中可以按照一定的顺序获取数据。
# 这在使用分布式训练时特别有用，可以确保各个进程获得的数据不会重复。
tokenized_dataset = tokenized_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/160 [00:00<?, ? examples/s]

In [11]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [12]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

### 加载模型

In [13]:
from transformers import AutoModel, AutoConfig, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# # QLoRA 量化配置
# # load_in_4bit=True 表示加载模型时使用4位量化。
# # bnb_4bit_quant_type='nf4' 指定了4位量化的类型。
# # bnb_4bit_use_double_quant=True 表示使用双重量化。
# # bnb_4bit_compute_dtype=_compute_dtype_map['bf16'] 使用了 _compute_dtype_map 中定义的数据类型 torch.bfloat16。

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244')

# 这行代码启用了模型的梯度检查点功能，这对于处理大型模型和长序列时内存效率很重要。
model.supports_gradient_checkpointing = True  

# 这行代码启用了梯度检查点，以在训练过程中节省内存。
model.gradient_checkpointing_enable()

# 这行代码启用了输入张量的梯度，允许对输入张量进行梯度计算。
model.enable_input_require_grads()

# 这行代码将模型配置中的 use_cache 设置为 False，以禁用缓存。这可以抑制一些警告信息，但在推断时应该重新启用缓存以获得更好的性能。
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

configuration_chatglm.py:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

modeling_chatglm.py:   0%|          | 0.00/55.7k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.2k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

...

(…)Y4G5oNnUVElA__&Key-Pair-Id=KCD77M1F0VK2B:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

(…)BO2yx00xS-sg__&Key-Pair-Id=KCD77M1F0VK2B:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

(…)UU81bAu0F1Ow__&Key-Pair-Id=KCD77M1F0VK2B:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

(…)EFpzrGNc3AtQ__&Key-Pair-Id=KCD77M1F0VK2B:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [14]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

# 这行代码使用 prepare_model_for_kbit_training 函数为知识增量训练（KBiT）准备模型。
# 变量 model 应该已经包含了您的基础模型，而 kbit_model 则会存储为进一步训练做好准备的模型。
kbit_model = prepare_model_for_kbit_training(model)

# 这行代码根据一个名为 TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING 的映射选择了训练的目标模块。
# 它似乎根据特定模型名称（在此为 'chatglm'）选择目标模块。
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [15]:
target_modules

['query_key_value']

In [16]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [17]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


### QLoRA 微调模型

In [18]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 3
output_dir = f"../../autodl-tmp/models/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=8,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [23]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [24]:
trainer.train()

Step,Training Loss
1,0.052400
2,0.047100
3,0.029100
4,0.022200
5,0.021800
6,0.020400
7,0.016400
8,0.018900
9,0.011200
10,0.014500


/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: U

TrainOutput(global_step=60, training_loss=0.008821041417346957, metrics={'train_runtime': 82.421, 'train_samples_per_second': 5.824, 'train_steps_per_second': 0.728, 'total_flos': 2430228439203840.0, 'train_loss': 0.008821041417346957, 'epoch': 3.0})

In [27]:
trainer.model.save_pretrained(output_dir)

## 微调前后效果对比

### 模型推理 - 使用微调前 ChatGLM3

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [6]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  # revision='b098244'
                                      )

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_chatglm.py:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- modeling_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True
                                          # ,revision='b098244', use_reentrant=True
                                         )

tokenization_chatglm.py:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [9]:
input_text = "解释下乾卦是什么？"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

乾卦是《易经》中的一个卦象，它是由两个阴爻夹一个阳爻构成，象征着天、干燥、强健、积极、刚毅等含义。乾卦是八卦之一，也是八宫图之一，位于北方。

乾卦的卦辞是“元、亨、利、贞”，这四个字意味着：元，表示开始、原始，亨，表示通、顺利，利，表示有利、吉祥，贞，表示正、正确。

乾卦的六爻分别有不同的含义：

初爻：阳爻，表示事物刚刚开始，有新的开始、机遇等含义。
二爻：阴爻，表示事物发展过程中遇到的困难和挫折。
三爻：阳爻，表示事物已经发展到了一半，开始出现积极、强健的迹象。
四爻：阴爻，表示事物已经到了一半，面临困难和挫折。
五爻：阳爻，表示事物已经发展到了最高峰，最富有、最强壮。
六爻：阴爻，表示事物已经到了最高峰，面临问题和危险。

乾卦的含义非常丰富，它既象征着阳刚之气、积极向上、强健有力，也象征着事物发展中的困难和挫折。因此，乾卦在《易经》中有着重要的地位，被认为是一种指导人们行为和决策的哲学思想。


In [10]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中的一个卦象，由两个阴爻夹一个阳爻构成，象征着地、水、军队等含义。地水师卦位于北方，属于八卦之一。

地水师卦的卦辞是“元、亨、利、贞”，与乾卦的卦辞相同。这四个字意味着：元，表示开始、原始，亨，表示通、顺利，利，表示有利、吉祥，贞，表示正、正确。

地水师卦的六爻分别有不同的含义：

初爻：阳爻，表示事物刚刚开始，有新的开始、机遇等含义。
二爻：阴爻，表示事物发展过程中遇到的困难和挫折。
三爻：阳爻，表示事物已经发展到了一半，开始出现积极、强健的迹象。
四爻：阴爻，表示事物已经到了一半，面临困难和挫折。
五爻：阳爻，表示事物已经发展到了最高峰，最富有、最强壮。
六爻：阴爻，表示事物已经到了最高峰，面临问题和危险。

地水师卦的含义丰富，它既象征着坚毅、勇敢、积极向上，也象征着事物发展中的困难和挫折。因此，地水师卦在《易经》中有着重要的地位，被认为是一种指导人们行为和决策的哲学思想。


### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [1]:
#配置缓存及网络环境，只需要开始时运行一次
import os
#在transformers自定义模型下载的路径方法
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["HF_DATASETS_CACHE"] = "../../autodl-tmp/datasets_cache/"
os.environ["HF_HOME"] = "../../autodl-tmp/cache/"
os.environ["HUGGINGFACE_HUB_CACHE"] = "../../autodl-tmp/hub_cache/"

#配置网络环境，只需要开始时运行一次

import subprocess
import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True, encoding='utf-8')

output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# 验证环境配置是否修改成功
print("http_proxy",os.environ.get("http_proxy"))
print("https_proxy",os.environ.get("https_proxy"))
print("HF_HOME",os.environ.get("HF_HOME"))
print("HF_DATASETS_CACHE",os.environ.get("HF_DATASETS_CACHE"))
print("HUGGINGFACE_HUB_CACHE",os.environ.get("HUGGINGFACE_HUB_CACHE"))

http_proxy http://172.20.0.113:12798
https_proxy http://172.20.0.113:12798
HF_HOME ../../autodl-tmp/cache/
HF_DATASETS_CACHE ../../autodl-tmp/datasets_cache/
HUGGINGFACE_HUB_CACHE ../../autodl-tmp/hub_cache/


In [2]:
from peft import PeftModel, PeftConfig
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
epochs = 3
timestamp = "20240209_103920"
peft_model_path = f"../../autodl-tmp/models/{model_name_or_path}-epoch{epochs}-{timestamp}"

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}


# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

base_model = AutoModel.from_pretrained('THUDM/chatglm3-6b',  # 模型ID或本地路径,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  # revision='b098244' 
                                      )

base_model.requires_grad_(False)
base_model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True,
                                          # revision='b098244', use_reentrant=True
                                         )

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- modeling_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
input_text = "解释下乾卦是什么？"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中最基本、最重要的卦象之一。乾卦代表天、代表刚强、代表行动、代表力量，具有积极向上、刚健不拔的品质。乾卦的卦象是由三个阳爻夹一个阴爻构成，象征天公无私、刚正不阿、积极向上、行动力强、力量大。

乾卦的卦辞是：“天行健，君子以自强不息。”这意味着，君子应该效仿天的行健，即不断前进，自强不息。同时，乾卦也提醒人们要有自信、有勇气、有决心，勇往直前，克服困难，迎接挑战。

在易经中，乾卦的卦义与八卦、五行、五行相呼應，具有丰富的象征意义。乾卦可以象征君主、领导、父亲、祖父等权威、力量、行动的人物。也可以象征天空、云彩、河流等自然现象，以及矿物、金属等自然物质。因此，乾卦具有广泛的应用和深刻的哲学意义。


In [4]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = peft_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [8]:
config = PeftConfig.from_pretrained(peft_model_path)
peft_model = PeftModel.from_pretrained(base_model,peft_model_path)

In [9]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [10]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个古老的卦象，在周易卦象中，它由上卦乾（天）和下卦坎（水）组成。坎象征着云，乾象征着天，云聚于天，形成了卦象。这个卦象预示着道德胜利，预示着吉兆。然而，胜利的喜悦不能持久，必须小心谨慎，适应环境，才能获得成功。在事业、经商、求名等方面，都需要谨慎行事，不可急功近利。地水师卦的核心哲学是：天时不如人时，厚德载物，适应环境，才能获得成功。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 在周易中，地水师卦是一个由地卦和 水卦组成的卦象，它预示着卜问者将得到卜卦师师的指示和指引。地卦代表着地象，它象征着稳定和坚实，而水卦则象征着流动和润泽。这一卦象预示着吉祥如意，吉利。

地水师卦的核心哲学是：稳扎稳打，厚积薄发。它鼓励人们坚定信念，踏实做事，不断积累经验，待时成熟，然后再采取行动。在这个卦象中，人们需要有耐心和恒心，不能急功近利。

地水师卦的核心意义：稳定坚实，逐步前进。它鼓励人们坚定信念，踏实做事，不断积累经验，待时成熟，然后再采取行动。这一卦象预示着吉祥如意，吉利。


In [11]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么卦象, 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。
